# Milestone 2
## 2.1 and 2.2 Split Train Dataset, Build and Evaluate a Model

In [2]:
import collections as c
import datetime
from IPython.core.interactiveshell import InteractiveShell
import matplotlib as mp
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import svm, tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm #creates progress bar to let you know how long is left till function is complete
import xgboost as xgb

InteractiveShell.ast_node_interactivity = 'all'

In [3]:
raw_data = pd.read_csv('../dataset/cases_train_processed.csv')

In [1]:
raw_data.isna().sum()

NameError: name 'raw_data' is not defined

In [4]:
# this way I dont have to load data all the time
df = raw_data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367634 entries, 0 to 367633
Data columns (total 23 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              367634 non-null  int64  
 1   index                   367634 non-null  int64  
 2   age                     367634 non-null  int64  
 3   sex                     367634 non-null  object 
 4   province                367634 non-null  object 
 5   country                 367634 non-null  object 
 6   date_confirmation       367634 non-null  object 
 7   additional_information  367634 non-null  object 
 8   source                  367634 non-null  object 
 9   outcome                 367634 non-null  object 
 10  Province_State          334812 non-null  object 
 11  Country_Region          356275 non-null  object 
 12  Last_Update             356275 non-null  object 
 13  Lat_right               356275 non-null  float64
 14  Long_right          

In [5]:
%%time
#remove nan values
df = df.drop(['Unnamed: 0','index','source','additional_information','Last_Update','Lat_right','Long_right','Province_State','Country_Region'],1)
df['Confirmed'].fillna(df['Confirmed'].mean(),inplace=True)
df['Deaths'].fillna(df.Deaths.mean(),inplace=True)
df['Recovered'].fillna(df.Recovered.mean(),inplace=True)
df['Active'].fillna(df.Active.mean(),inplace=True)
df['Incidence_Rate'].fillna(df.Incidence_Rate.mean(),inplace=True)
df['Case-Fatality_Ratio'].fillna(df['Case-Fatality_Ratio'].mean(),inplace=True)
df.date_confirmation = pd.to_datetime(df.date_confirmation,infer_datetime_format=True) 
df.Combined_Key.fillna((df.province+" ,"+df.country),inplace=True)


Wall time: 313 ms


In [19]:
df.isna().sum()

age                    0
sex                    0
province               0
country                0
date_confirmation      0
outcome                0
Confirmed              0
Deaths                 0
Recovered              0
Active                 0
Combined_Key           0
Incidence_Rate         0
Case-Fatality_Ratio    0
dist_between_in_km     0
dtype: int64

# Exploratory Analysis

In [ ]:
# exploratory analysis
pie, ax = plt.subplots(figsize=[10,6])
labels = countries.keys()
plt.pie(x=countries, autopct="%.1f%%", explode=[0.05]*len(countries), labels=labels, pctdistance=0.8)
plt.title("Top 10 Countries", fontsize=14);

country_list = list(countries.keys())

def fix_values(row):
        if row['country'] not in country_list:
            row['country'] = 'Other'
        return row 

df_other = df.apply(fix_values, axis=1)
countries_other = df_other['country'].value_counts()

pie, ax = plt.subplots(figsize=[10,6])
labels = countries_other.keys()
plt.pie(x=countries_other, autopct="%.1f%%", explode=[0.05]*len(countries_other), labels=labels, pctdistance=0.85)
plt.title("Top 10 Countries (Including others)", fontsize=14);

In [ ]:
pie, ax = plt.subplots(figsize=[10,6])
labels = 

In [6]:
df.outcome = df.outcome.astype('string')


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367634 entries, 0 to 367633
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   age                  367634 non-null  int64         
 1   sex                  367634 non-null  object        
 2   province             367634 non-null  object        
 3   country              367634 non-null  object        
 4   date_confirmation    367634 non-null  datetime64[ns]
 5   outcome              367634 non-null  string        
 6   Confirmed            367634 non-null  float64       
 7   Deaths               367634 non-null  float64       
 8   Recovered            367634 non-null  float64       
 9   Active               367634 non-null  float64       
 10  Combined_Key         367634 non-null  object        
 11  Incidence_Rate       367634 non-null  float64       
 12  Case-Fatality_Ratio  367634 non-null  float64       
 13  dist_between_i

In [8]:
# once hot encode and add new cols to dataframe
def oneHotEncode_df(dataframe):
    col2Encode = list(df.select_dtypes(include=['object']))
    dummies = pd.get_dummies(dataframe,columns=col2Encode,prefix=col2Encode)
    res = pd.concat([dataframe, dummies], axis=1)
    #if we decide to drop one hot encoded values
    res = res.drop(col2Encode, axis=1)
    return(res)

In [9]:
%%time
# add dummy variables  
ohe_df = oneHotEncode_df(df)
ohe_df

Wall time: 21.2 s


,age,date_confirmation,outcome,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio,dist_between_in_km,...,"Combined_Key_Zamboanga del Sur ,Philippines","Combined_Key_Zamfara ,Nigeria","Combined_Key_Zanjan ,Iran","Combined_Key_Zeeland, Netherlands","Combined_Key_Zhejiang, China",Combined_Key_Zimbabwe,"Combined_Key_Zuid-Holland, Netherlands","Combined_Key_galapagos ,Ecuador","Combined_Key_Ñeembucu ,Paraguay","Combined_Key_ñeembucu ,Paraguay"
0,7,2020-05-26,recovered,238828.000000,4907.000000,201671.000000,32250.000000,1276.409575,2.054617,10.390897,...,0,0,0,0,0,0,0,0,0,0
1,7,2020-05-20,hospitalized,342788.000000,4869.000000,270094.000000,67825.000000,144.099577,1.420411,209.514800,...,0,0,0,0,0,0,0,0,0,0
2,7,2020-05-26,hospitalized,2859.000000,2.000000,2639.000000,218.000000,464.331421,0.069955,204.687238,...,0,0,0,0,0,0,0,0,0,0
3,3,2020-03-15,nonhospitalized,46779.000000,1871.000000,41228.000000,3680.000000,422.592353,3.999658,126.061115,...,0,0,0,0,0,0,0,0,0,0
4,7,2020-05-20,hospitalized,120336.000000,3286.000000,100974.000000,16076.000000,188.400627,2.730687,217.585376,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367629,3,2020-03-31,nonhospitalized,106331.902334,2739.064287,77838.956438,25753.923921,968.698456,2.721038,-1.000000,...,0,0,0,0,0,0,0,0,0,0
367630,7,2020-04-16,hospitalized,238828.000000,4907.000000,201671.000000,32250.000000,1276.409575,2.054617,10.390897,...,0,0,0,0,0,0,0,0,0,0
367631,7,2020-05-30,hospitalized,2859.000000,2.000000,2639.000000,218.000000,464.331421,0.069955,111.254821,...,0,0,0,0,0,0,0,0,0,0
367632,3,2020-03-02,recovered,57558.000000,27.000000,57142.000000,389.000000,983.839751,0.046909,0.367129,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#split into dependent and independent var
x = ohe_df.copy().drop(columns=['outcome','date_confirmation','dist_between_in_km'])
y = ohe_df.outcome.copy()

In [18]:
t_x,v_x,t_y, v_y = train_test_split(x,y, train_size=0.8, random_state=1)

In [20]:

v_y
t_x

,outcome,outcome
247886,nonhospitalized,nonhospitalized
364290,nonhospitalized,nonhospitalized
237307,hospitalized,hospitalized
22967,hospitalized,hospitalized
29696,hospitalized,hospitalized
...,...,...
47882,hospitalized,hospitalized
319748,nonhospitalized,nonhospitalized
153914,nonhospitalized,nonhospitalized
219937,nonhospitalized,nonhospitalized


,age,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio,age,Confirmed,Deaths,...,"Combined_Key_Zamboanga del Sur ,Philippines","Combined_Key_Zamfara ,Nigeria","Combined_Key_Zanjan ,Iran","Combined_Key_Zeeland, Netherlands","Combined_Key_Zhejiang, China",Combined_Key_Zimbabwe,"Combined_Key_Zuid-Holland, Netherlands","Combined_Key_galapagos ,Ecuador","Combined_Key_Ñeembucu ,Paraguay","Combined_Key_ñeembucu ,Paraguay"
218247,3,2531.0,76.0,1571.0,884.0,468.787044,3.002766,3,2531.0,76.0,...,0,0,0,0,0,0,0,0,0,0
33854,7,120336.0,3286.0,100974.0,16076.0,188.400627,2.730687,7,120336.0,3286.0,...,0,0,0,0,0,0,0,0,0,0
234440,7,238828.0,4907.0,201671.0,32250.0,1276.409575,2.054617,7,238828.0,4907.0,...,0,0,0,0,0,0,0,0,0,0
316468,7,21913.0,462.0,16715.0,4736.0,1550.219236,2.108338,7,21913.0,462.0,...,0,0,0,0,0,0,0,0,0,0
250495,4,25278.0,1546.0,0.0,23732.0,2592.083675,6.115990,4,25278.0,1546.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117583,3,21913.0,462.0,16715.0,4736.0,1550.219236,2.108338,3,21913.0,462.0,...,0,0,0,0,0,0,0,0,0,0
73349,6,64692.0,1844.0,58524.0,4324.0,360.749785,2.850430,6,64692.0,1844.0,...,0,0,0,0,0,0,0,0,0,0
312201,7,2859.0,2.0,2639.0,218.0,464.331421,0.069955,7,2859.0,2.0,...,0,0,0,0,0,0,0,0,0,0
267336,7,2859.0,2.0,2639.0,218.0,464.331421,0.069955,7,2859.0,2.0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
%%time
#split train data
trainData, validationData = train_test_split(ohe_df, train_size=0.8, random_state=1)

Wall time: 7.09 s


In [27]:
#split dataset into dependent and independent var
validationData_x = validationData.copy().drop(columns=['outcome','date_confirmation','dist_between_in_km'])
validationData_y = validationData.outcome.copy()

trainData_x = trainData.copy().drop(columns=['outcome','date_confirmation','dist_between_in_km'])
trainData_y = trainData.outcome.copy()

In [28]:
trainData_y.iloc[:,1]
validationData_x

218247    nonhospitalized
33854           recovered
234440       hospitalized
316468       hospitalized
250495    nonhospitalized
               ...       
117583       hospitalized
73349     nonhospitalized
312201       hospitalized
267336       hospitalized
128037          recovered
Name: outcome, Length: 294107, dtype: string

,age,Confirmed,Deaths,Recovered,Active,Incidence_Rate,Case-Fatality_Ratio,age,Confirmed,Deaths,...,"Combined_Key_Zamboanga del Sur ,Philippines","Combined_Key_Zamfara ,Nigeria","Combined_Key_Zanjan ,Iran","Combined_Key_Zeeland, Netherlands","Combined_Key_Zhejiang, China",Combined_Key_Zimbabwe,"Combined_Key_Zuid-Holland, Netherlands","Combined_Key_galapagos ,Ecuador","Combined_Key_Ñeembucu ,Paraguay","Combined_Key_ñeembucu ,Paraguay"
247886,3,11703.000000,253.000000,0.000000,11450.000000,6073.170732,2.161839,3,11703.000000,253.000000,...,0,0,0,0,0,0,0,0,0,0
364290,4,106331.902334,2739.064287,77838.956438,25753.923921,968.698456,2.721038,4,106331.902334,2739.064287,...,0,0,0,0,0,0,0,0,0,0
237307,4,21913.000000,462.000000,16715.000000,4736.000000,1550.219236,2.108338,4,21913.000000,462.000000,...,0,0,0,0,0,0,0,0,0,0
22967,7,530908.000000,8685.000000,475717.000000,46506.000000,682.039258,1.635877,7,530908.000000,8685.000000,...,0,0,0,0,0,0,0,0,0,0
29696,7,111290.000000,1308.000000,92265.000000,17717.000000,137.339636,1.175308,7,111290.000000,1308.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47882,7,2859.000000,2.000000,2639.000000,218.000000,464.331421,0.069955,7,2859.000000,2.000000,...,0,0,0,0,0,0,0,0,0,0
319748,4,11703.000000,253.000000,0.000000,11450.000000,6073.170732,2.161839,4,11703.000000,253.000000,...,0,0,0,0,0,0,0,0,0,0
153914,4,24711.000000,1650.000000,0.000000,23061.000000,1885.184620,6.677188,4,24711.000000,1650.000000,...,0,0,0,0,0,0,0,0,0,0
219937,5,7149.000000,269.000000,6129.000000,751.000000,388.283812,3.762764,5,7149.000000,269.000000,...,0,0,0,0,0,0,0,0,0,0


# ----------------------------------Model Building-----------------------------------

In [30]:
#create 3 diff classififers and append them to list
classifiers = []

xgb_model = xgb.XGBClassifier()
classifiers.append(xgb_model)

randForest_model = RandomForestClassifier()
classifiers.append(randForest_model)

svm_model = svm.SVC()
classifiers.append(svm_model)

In [44]:
a = c.Counter(df.columns)
df.sum()

Error: ModuleNotFoundError

In [31]:
#test models

for clas in tqdm(classifiers):
    clas.fit(trainData_x,trainData_y.iloc[:,0])
    predicted = clas.predict(validationData_x)
    accuracy = accuracy_score(validationData_y.iloc[:,0],predicted)
    print("Accuracy of %s is %s"%(clas, accuracy))
    con_mat = confusion_matrix(validationData_y.iloc[:,0],predicted)
    print("Confusion Matrix of %s is %s"%(clas, con_mat))

  0%|          | 0/3 [00:00<?, ?it/s]c:\Users\I537205\Documents\C459\DoR459_Proj\venv\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
  0%|          | 0/3 [01:33<?, ?it/s]


ValueError: feature_names must be unique